In [49]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
from pprint import pprint

In [2]:
class feature_weight :
    def __init__(self) :
        iris = datasets.load_iris()
        self.data, self.target = pd.DataFrame(iris.data, columns=iris.feature_names), iris.target

    def global_accuracy(self) :
        """
        Calculate Global Knn accuracy
        """
        global_X_train, global_X_val, global_y_train, global_y_val = train_test_split(self.data, self.target)
    
        knn = KNeighborsClassifier(n_neighbors=5)
        knn.fit(global_X_train, global_y_train)
        self.original_acc = accuracy_score(knn.predict(global_X_val), global_y_val)
        
        
    def feature_iteration(self) :
        """
        Calculate each accuarcy that remove one column
        """
        self.global_accuracy()
        self.weight = []
        
        for col in self.data :
            #remove one column
            column = list(self.data.columns)
            column.remove(col)
            
            #make new dataset that remove one column
            adjusted_data = self.data[column]
            X_train, X_val, y_train, y_val = train_test_split(adjusted_data, self.target)
            
            #learning using Knn
            knn = KNeighborsClassifier(n_neighbors=5)
            knn.fit(X_train, y_train)
            self.weight.append(accuracy_score(knn.predict(X_val), y_val))
            
    def get_weight(self) :
        """
        Calculate weight using paper
        """
        self.feature_iteration()
        for idx in range(len(self.weight)) :
            self.weight[idx] = 1-(self.weight[idx]-self.original_acc)
            
        sum_of_disc = sum(self.weight)
        for idx in range(len(self.weight)) :
            self.weight[idx] = self.weight[idx]/sum_of_disc
            
        return self.weight

In [18]:
iris = datasets.load_iris()
data, target = pd.DataFrame(iris.data, columns=iris.feature_names), iris.target

# Calculate global accuracy

In [30]:
X_train, X_val, y_train, y_val = train_test_split(data, target)

In [31]:
k_list = [1,2,3,4,5]
global_acc = []

for k in k_list :
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)
  global_acc.append(accuracy_score(knn.predict(X_val), y_val))

In [32]:
global_acc

[0.868421052631579,
 0.8947368421052632,
 0.8947368421052632,
 0.9210526315789473,
 0.9210526315789473]

# Weighted Euclidean distance

In [7]:
def weigthed_l2(weight, row1, row2) :
  sub = row1-row2
  return np.sqrt((weight*sub*sub).sum())

In [8]:
def normal_l2(row1, row2) :
  sub = row1-row2
  return np.sqrt((sub*sub).sum())

In [9]:
normal_l2(data.iloc[0, :], data.iloc[1, :])

0.5385164807134502

In [33]:
weight = feature_weight().get_weight()
weigthed_l2(weight, data.iloc[0, :], data.iloc[1, :])

0.26917063396418117

**계산기로 계산해본 결과 위 결과와 동일** 

# 각 Point별로 weighted euclidean distance 구하기

In [34]:
dist_matrix = np.zeros((len(target), len(target)))

mat_data = np.array(data)
for idx_0, value_0 in enumerate(mat_data) :
  for idx_1, value_1 in enumerate(mat_data) :
    dist_matrix[idx_0, idx_1] = weigthed_l2(weight, value_0, value_1)

In [ ]:
# for test
# Do not need to run this cell
idx_list = []
value_list = []
for idx, value in enumerate(dist_matrix[101]) :
  idx_list.append(idx)
  value_list.append(value)

sort_df = pd.DataFrame((idx_list, value_list), index=['index', 'value']).T
sort_df = sort_df.sort_values(by=['value'])
sorted_index_list = list(sort_df.iloc[1:k_list[4]+1, :].index)

print(target[101]==[target[i] for i in sorted_index_list])
print((target[101]==[target[i] for i in sorted_index_list]).mean())

[ True  True  True  True False]
0.8


In [35]:
local_k_df = []
local_k_list = []

for k in k_list :
  acc_list = []

  for i , row in enumerate(dist_matrix) :
    idx_list = []
    value_list = []
    
    for idx, value in enumerate(row) :
      idx_list.append(idx)
      value_list.append(value)

    sort_df = pd.DataFrame((idx_list, value_list), index=['index', 'value']).T
    sort_df = sort_df.sort_values(by=['value'])
    sort_df = list(sort_df.iloc[1:k+1, :].index)

    acc_list.append((target[i]== [target[j] for j in sort_df]).mean())
  local_k_df.append(acc_list+global_acc[k-1])

In [36]:
local_k_df = np.array(local_k_df).T
local_k_df = pd.DataFrame(local_k_df, columns=['k=1', 'k=2', 'k=3', 'k=4', 'k=5'])
local_k_df

,k=1,k=2,k=3,k=4,k=5
0,1.868421,1.894737,1.894737,1.921053,1.921053
1,1.868421,1.894737,1.894737,1.921053,1.921053
2,1.868421,1.894737,1.894737,1.921053,1.921053
3,1.868421,1.894737,1.894737,1.921053,1.921053
4,1.868421,1.894737,1.894737,1.921053,1.921053
...,...,...,...,...,...
145,1.868421,1.894737,1.894737,1.921053,1.921053
146,1.868421,1.894737,1.894737,1.671053,1.521053
147,1.868421,1.894737,1.894737,1.921053,1.921053
148,1.868421,1.894737,1.894737,1.921053,1.921053


# Determine k
train test shuffle  
150개의 30%인 45개의 validation 데이터 생성

In [70]:
random.seed(0)
val_idx = random.sample(list(local_k_df.index), int(len(local_k_df)*0.3))

train_idx = list(local_k_df.index)
for idx in val_idx :
  train_idx.remove(idx)

In [73]:
 train_local_k_df, test_local_k_df = local_k_df.iloc[train_idx, :], local_k_df.iloc[val_idx, :]

In [75]:
train_local_k_df

,k=1,k=2,k=3,k=4,k=5
0,1.868421,1.894737,1.894737,1.921053,1.921053
2,1.868421,1.894737,1.894737,1.921053,1.921053
3,1.868421,1.894737,1.894737,1.921053,1.921053
4,1.868421,1.894737,1.894737,1.921053,1.921053
5,1.868421,1.894737,1.894737,1.921053,1.921053
...,...,...,...,...,...
144,1.868421,1.894737,1.894737,1.921053,1.921053
145,1.868421,1.894737,1.894737,1.921053,1.921053
147,1.868421,1.894737,1.894737,1.921053,1.921053
148,1.868421,1.894737,1.894737,1.921053,1.921053


In [92]:
train_local_k_df['fitted_k'] = 0

for idx, row in train_local_k_df.iterrows() :
  row = list(row)
  train_local_k_df.loc[idx, 'fitted_k'] = row.index(max(row))+1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [93]:
train_local_k_df

,k=1,k=2,k=3,k=4,k=5,fitted_k
0,1.868421,1.894737,1.894737,1.921053,1.921053,4
2,1.868421,1.894737,1.894737,1.921053,1.921053,4
3,1.868421,1.894737,1.894737,1.921053,1.921053,4
4,1.868421,1.894737,1.894737,1.921053,1.921053,4
5,1.868421,1.894737,1.894737,1.921053,1.921053,4
...,...,...,...,...,...,...
144,1.868421,1.894737,1.894737,1.921053,1.921053,4
145,1.868421,1.894737,1.894737,1.921053,1.921053,4
147,1.868421,1.894737,1.894737,1.921053,1.921053,4
148,1.868421,1.894737,1.894737,1.921053,1.921053,4


In [98]:
train_local_k_df['fitted_k'].value_counts()

4    98
2     3
5     2
1     2
Name: fitted_k, dtype: int64